In [16]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString, MultiLineString

# 파일 경로 설정
csv_path = "./testing/광진구_clusters_route 1.csv"
geojson_path = "./testing/road.geojson"

# CSV 데이터 로드
df = pd.read_csv(csv_path)

# 도로 네트워크 로드
gdf_roads = gpd.read_file(geojson_path)

# 도로 네트워크 그래프 생성
G = nx.Graph()

# GeoJSON에서 도로 데이터 추가 (MultiLineString 포함)
for _, row in gdf_roads.iterrows():
    geom = row.geometry
    if isinstance(geom, MultiLineString):
        for line in geom.geoms:
            coords = list(line.coords)
            for i in range(len(coords) - 1):
                G.add_node(coords[i], pos=coords[i])
                G.add_node(coords[i + 1], pos=coords[i + 1])
                G.add_edge(coords[i], coords[i + 1], weight=line.length)
    elif isinstance(geom, LineString):
        coords = list(geom.coords)
        for i in range(len(coords) - 1):
            G.add_node(coords[i], pos=coords[i])
            G.add_node(coords[i + 1], pos=coords[i + 1])
            G.add_edge(coords[i], coords[i + 1], weight=geom.length)

# 노드 개수 확인
if len(G.nodes) == 0:
    raise ValueError("도로 네트워크가 생성되지 않았습니다. road.geojson 데이터를 확인하세요.")

# 0번 클러스터만 필터링
df_cluster_0 = df[df['cluster'] == 0].sort_values(by='order')

# 클러스터 0의 좌표 리스트 (lat, lon 순서)
coordinates = list(zip(df_cluster_0['latitude'], df_cluster_0['longitude']))

# 도로 위의 가장 가까운 노드 찾기
nodes = []
for lat, lon in coordinates:
    closest_node = min(G.nodes, key=lambda node: (node[0] - lon)**2 + (node[1] - lat)**2)
    nodes.append(closest_node)

# 최적 경로 찾기 (도로 네트워크 기반)
route_edges = []
for i in range(len(nodes) - 1):
    path = nx.shortest_path(G, source=nodes[i], target=nodes[i + 1], weight="weight")
    edges = [(path[j], path[j + 1]) for j in range(len(path) - 1)]
    for edge in edges:
        route_edges.append(LineString([edge[0], edge[1]]))

# 경로 데이터를 GeoDataFrame으로 변환
gdf_route = gpd.GeoDataFrame(geometry=route_edges, crs=gdf_roads.crs)

# GeoJSON 파일로 저장
gdf_route.to_file("cluster_0_route.geojson", driver="GeoJSON")

print("GeoJSON 저장 완료: cluster_0_route.geojson")


GeoJSON 저장 완료: cluster_0_route.geojson


In [17]:
gdf_route.head()

,geometry
0,"LINESTRING (127.09402 37.53794, 127.09378 37.5..."
1,"LINESTRING (127.09378 37.53803, 127.09368 37.5..."
2,"LINESTRING (127.09368 37.53807, 127.09359 37.5..."
3,"LINESTRING (127.09359 37.53792, 127.09341 37.5..."
4,"LINESTRING (127.09341 37.53765, 127.09333 37.5..."


In [18]:
import folium
import geopandas as gpd

# GeoJSON 파일 로드
geojson_path = "cluster_0_route.geojson"
gdf_route = gpd.read_file(geojson_path)

# 지도 생성 (초기 중심은 첫 번째 좌표 기준)
center = [gdf_route.geometry.iloc[0].centroid.y, gdf_route.geometry.iloc[0].centroid.x]
m = folium.Map(location=center, zoom_start=15)

# GeoJSON 경로 추가
folium.GeoJson(gdf_route, name="Route").add_to(m)

# HTML 파일로 저장
m.save("cluster_0_route_map.html")

# 출력 메시지
print("HTML 지도 저장 완료: cluster_0_route_map.html")

HTML 지도 저장 완료: cluster_0_route_map.html


In [12]:
import os
print(os.getcwd())  # 현재 작업 디렉토리 확인


c:\Users\User\Desktop\빅프로젝트\BigProject\visualization


In [1]:
# import geopandas as gpd

# # legal_boundary 파일 로드
# legal_boundary_path = "../processed_geojson_data/legal_boundary.geojson"
# legal_boundary = gpd.read_file(legal_boundary_path)

# # 광진구 경계 추출
# gwangjin_boundary = legal_boundary[legal_boundary["SIG_KOR_NM"] == "광진구"]

# # sidewalk 파일 로드 (경로 변경 필요)
# sidewalk_path = "../meaningful_data/sidewalk.geojson"  # 실제 파일 경로로 변경
# sidewalk = gpd.read_file(sidewalk_path)
# # sidewalk.head()
# # 광진구 경계 내의 sidewalk 데이터 필터링
# sidewalk_gwangjin = gpd.overlay(sidewalk, gwangjin_boundary, how="intersection")

# sidewalk_gwangjin = sidewalk_gwangjin.dissolve().explode(index_parts=False)
# sidewalk_gwangjin["geometry"] = sidewalk_gwangjin.geometry.apply(
#     lambda x: MultiLineString([x]) if x.geom_type == "LineString" else x
# )
# sidewalk_gwangjin.to_file(output_path, driver="GeoJSON")
# # 결과 저장
# output_path = "sidewalk_gwangjin.geojson"
# sidewalk_gwangjin.to_file(output_path, driver="GeoJSON")

# print(f"광진구 내 sidewalk 데이터 저장 완료: {output_path}")



광진구 내 sidewalk 데이터 저장 완료: sidewalk_gwangjin.geojson


In [8]:
import geopandas as gpd
from shapely.geometry import MultiLineString

# legal_boundary 파일 로드
legal_boundary_path = "../processed_geojson_data/legal_boundary.geojson"
legal_boundary = gpd.read_file(legal_boundary_path)

# 서울 전체 경계 추출
seoul_boundary = legal_boundary.dissolve()  # 서울 전체를 하나의 경계로 합치기

# sidewalk 파일 로드 (경로 변경 필요)
sidewalk_path = "../meaningful_data/sidewalk.geojson"  # 실제 파일 경로로 변경
sidewalk = gpd.read_file(sidewalk_path)

# 서울 경계 내 sidewalk 데이터 필터링
sidewalk_seoul = gpd.overlay(sidewalk, seoul_boundary, how="intersection")

# LineString을 MultiLineString으로 변환
sidewalk_seoul = sidewalk_seoul.dissolve().explode(index_parts=False)
sidewalk_seoul["geometry"] = sidewalk_seoul.geometry.apply(
    lambda x: MultiLineString([x]) if x.geom_type == "LineString" else x
)

# 결과 저장
output_path = "sidewalk_seoul.geojson"
sidewalk_seoul.to_file(output_path, driver="GeoJSON")

print(f"서울 내 sidewalk 데이터 저장 완료: {output_path}")
display(sidewalk_seoul.head())


서울 내 sidewalk 데이터 저장 완료: sidewalk_seoul.geojson


,UFID,WIDT,QUAL,BYYN,KIND,SCLS,FMTA,geometry_type,coordinates,SIG_CD,SIG_KOR_NM,geometry
0,100037612020A0031ebc2f1560e3f4080,0.0,SWQ000,BYC002,SWK001,A0033324,S2116746,MultiLineString,"[ [ [ 126.98595448186775, 37.468262635749007 ]...",11110,종로구,"MULTILINESTRING ((126.98595 37.46826, 126.9859..."
0,100037612020A0031ebc2f1560e3f4080,0.0,SWQ000,BYC002,SWK001,A0033324,S2116746,MultiLineString,"[ [ [ 126.98595448186775, 37.468262635749007 ]...",11110,종로구,"MULTILINESTRING ((126.88609 37.46996, 126.8860..."
0,100037612020A0031ebc2f1560e3f4080,0.0,SWQ000,BYC002,SWK001,A0033324,S2116746,MultiLineString,"[ [ [ 126.98595448186775, 37.468262635749007 ]...",11110,종로구,"MULTILINESTRING ((126.90615 37.47273, 126.9061..."
0,100037612020A0031ebc2f1560e3f4080,0.0,SWQ000,BYC002,SWK001,A0033324,S2116746,MultiLineString,"[ [ [ 126.98595448186775, 37.468262635749007 ]...",11110,종로구,"MULTILINESTRING ((126.90623 37.47239, 126.9062..."
0,100037612020A0031ebc2f1560e3f4080,0.0,SWQ000,BYC002,SWK001,A0033324,S2116746,MultiLineString,"[ [ [ 126.98595448186775, 37.468262635749007 ]...",11110,종로구,"MULTILINESTRING ((126.90642 37.47156, 126.9064..."


In [10]:
import matplotlib.font_manager as fm

# 나눔글꼴 경로 설정
font_path = 'C:\Windows\Fonts\malgun.ttf'

# 폰트 이름 가져오기
font_name = fm.FontProperties(fname=font_path).get_name()

# 폰트 설정
plt.rc('font', family=font_name)

<>:4: SyntaxWarning: invalid escape sequence '\W'
<>:4: SyntaxWarning: invalid escape sequence '\W'
C:\Users\User\AppData\Local\Temp\ipykernel_19828\3008152010.py:4: SyntaxWarning: invalid escape sequence '\W'
  font_path = 'C:\Windows\Fonts\malgun.ttf'


In [15]:
import folium
import geopandas as gpd
from folium.plugins import Fullscreen

# 서울 인도 데이터 로드
sidewalk_seoul_path = "./testing/sidewalk_seoul.geojson"  # 저장된 파일 경로
sidewalk_seoul = gpd.read_file(sidewalk_seoul_path)

# 서울 도로 데이터 로드
road_path = "./testing/road.geojson"
road = gpd.read_file(road_path)

# 서울 중심 좌표 (대략적인 중심점)
seoul_center = [37.5665, 126.9780]

# Folium 지도 생성
m = folium.Map(location=seoul_center, zoom_start=12, tiles="Stamen Toner")

# 인도 데이터 추가 (하늘색) → 그룹: "인도"
sidewalk_layer = folium.GeoJson(
    sidewalk_seoul,
    name="서울 인도 데이터",
    style_function=lambda feature: {
        "color": "skyblue",
        "weight": 1,  # 선 굵기 조정
        "opacity": 1,
    },
).add_to(m)

# 도로 데이터 추가 (핑크색) → 그룹: "도로"
road_layer = folium.GeoJson(
    road,
    name="서울 도로 데이터",
    style_function=lambda feature: {
        "color": "pink",
        "weight": 1.5,  # 선 굵기 조정
        "opacity": 1,
    },
).add_to(m)

# **레이어 컨트롤 추가 (선택 기능)**
folium.LayerControl(collapsed=False).add_to(m)  # collapsed=False → 펼쳐진 상태로 보이게 설정

# 전체화면 버튼 추가
Fullscreen().add_to(m)

# 지도 저장 (Jupyter Notebook에서 볼 경우)
map_path = "seoul_sidewalk_map3.html"
m.save(map_path)

print(f"인터랙티브 지도 저장 완료: {map_path}")



인터랙티브 지도 저장 완료: seoul_sidewalk_map3.html
